This notebook compares analytical and numerical evaluations of alchemical derivatives of a CO molecule up to the third order in Hartree-Fock theory (HFT).  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf

pyscf.__version__

'2.2.1'

In [2]:
from aqa.FcMole import FcM_like

In [3]:
target_mol = "C 0 0 0; O 0 0 2.1"
# dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"

### Analytical evaluation

In [4]:
%load_ext autoreload
from aqa.AP_class import APDFT_perturbator as AP

In [5]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1],flag_response_property=True)

converged SCF energy = -112.786968394487


In [6]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force is not implemented in the Hartree-Fock theory

Inefficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [7]:
# inefficient_analytical_alchemical_force = np.zeros((2, 2, 3))
# inefficient_analytical_alchemical_force[0] = ap_nn.build_inefficient_alchemical_force(0)
# inefficient_analytical_alchemical_force[1] = ap_nn.build_inefficient_alchemical_force(1)

Efficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [8]:
efficient_analytical_alchemical_force = np.zeros((2, 2, 3))
efficient_analytical_alchemical_force[0] = ap_nn.build_alchemical_force(0)
efficient_analytical_alchemical_force[1] = ap_nn.build_alchemical_force(1)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [9]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [10]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [11]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0]
fcs2 = [-fc_param, 0.0]
fcs3 = [0.0, fc_param]
fcs4 = [0.0, -fc_param]

##### Efficient numerical alchemical force

In [12]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
mf=scf.RHF(mol_NN)
mf1=scf.RHF(fmol1)
mf2=scf.RHF(fmol2)
mf3=scf.RHF(fmol3)
mf4=scf.RHF(fmol4)
# mf.xc = dft_functional
# mf1.xc = dft_functional
# mf2.xc = dft_functional
# mf3.xc = dft_functional
# mf4.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())

converged SCF energy = -112.786968394493


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -112.801621815929
converged SCF energy = -112.772317855411
converged SCF energy = -112.809217062771
converged SCF energy = -112.764723307082


In [13]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0232743825
1 O    -0.0000000000    -0.0000000000     0.0232743825
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0235424322
1 O    -0.0000000000    -0.0000000000     0.0235424322
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0230063239
1 O    -0.0000000000    -0.0000000000     0.0230063239
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0233207040
1 O    -0.0000000000    -0.0000000000     0.0233207040
----------------------------------------------
--------

In [14]:
efficient_numerical_alchemical_force = np.zeros((2, 2, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)

#### Numerical alchemical derivatives

In [15]:
numerical_1st_alchemical_derivative = np.zeros((2))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)

#### Numerical alchemical hardness

In [16]:
ap_nn=AP(mf,sites=[0,1])
ap_nn1=AP(mf1,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn2=AP(mf2,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn3=AP(mf3,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn4=AP(mf4,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1

In [17]:
numerical_2nd_alchemical_derivative = np.zeros((2, 2))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [18]:
numerical_3rd_alchemical_derivative = np.zeros((2, 2, 2))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [19]:
mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
mf_nn1=scf.RHF(mol_NN1)
mf_nn2=scf.RHF(mol_NN2)
mf_nn3=scf.RHF(mol_NN3)
mf_nn4=scf.RHF(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
mf_nn1.scf()
mf_nn2.scf()
mf_nn3.scf()
mf_nn4.scf()
ap_nn1=AP(mf_nn1,sites=[0,1])
ap_nn2=AP(mf_nn2,sites=[0,1])
ap_nn3=AP(mf_nn3,sites=[0,1])
ap_nn4=AP(mf_nn4,sites=[0,1])
ad1 = ap_nn1.build_gradient()
ad2 = ap_nn2.build_gradient()
ad3 = ap_nn3.build_gradient()
ad4 = ap_nn4.build_gradient()

converged SCF energy = -112.786990945021
converged SCF energy = -112.786944398205
converged SCF energy = -112.786944398205
converged SCF energy = -112.786990945021


In [20]:
inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [21]:
numer_elec_electric_dipole_gradient = np.zeros((2, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [22]:
numer_elec_electric_pol_gradient = np.zeros((2, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [23]:
print(analytical_1st_alchemical_derivative)

[-14.65197969 -22.24687958]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [24]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 5.63956345e-07 -1.73739907e-06]


$\partial^2E/\partial Z_i\partial Z_j$

In [25]:
print(analytical_2nd_alchemical_derivative)

[[-2.88235455  0.46095804]
 [ 0.46095804 -3.58086845]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [26]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 4.81800739e-07 -4.12107297e-07]
 [-1.25736754e-10 -3.49784202e-07]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [27]:
print(analytical_3rd_alchemical_derivative)

[[[-0.13175055  0.13763008]
  [ 0.13763008  0.08208886]]

 [[ 0.13763008  0.08208886]
  [ 0.08208886 -0.16691396]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [28]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 2.59153046e-07 -1.11076166e-08]
  [-1.11076166e-08 -1.51670726e-07]]

 [[-6.20978287e-08 -1.97341526e-07]
  [-1.97341526e-07  3.51903934e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [29]:
print(efficient_analytical_alchemical_force)

[[[ 5.14823433e-16  2.59283230e-17 -2.68054376e-01]
  [-5.14823433e-16 -2.59283230e-17  2.68054376e-01]]

 [[ 4.98716527e-16  3.03802737e-16 -4.64206327e-02]
  [-4.98716527e-16 -3.03802737e-16  4.64206327e-02]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [30]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[-1.81509306e-14 -1.35059886e-13 -1.93717221e-07]
  [ 1.81509306e-14  1.35059886e-13  1.93722551e-07]]

 [[ 7.95050152e-14 -7.34300003e-14  1.55680352e-07]
  [-7.95050152e-14  7.34300003e-14 -1.55667931e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [31]:
# print(inefficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [32]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [33]:
print(elec_electric_dipole_gradient_f)

[[-2.93587572e-15 -2.21038261e-15 -5.89888119e-01]
 [-1.78037332e-15 -3.13152568e-15  1.26555184e+00]]


In [34]:
print(elec_electric_dipole_gradient_z)

[[-2.93557880e-15 -2.21022154e-15 -5.89888113e-01]
 [-1.78014846e-15 -3.13143689e-15  1.26555189e+00]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [35]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[ 9.64415267e-13  6.80454027e-14  1.17360157e-06]
 [ 1.35820103e-12 -1.15482215e-12 -2.98897690e-06]]


In [36]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[ 9.64415564e-13  6.80455638e-14  1.17943105e-06]
 [ 1.35820125e-12 -1.15482206e-12 -2.94012188e-06]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [37]:
print(elec_electric_pol_gradient)

[[[ 7.31927487e+00 -2.39396495e-14 -3.35486434e-14]
  [-2.39396495e-14  7.31927487e+00 -3.83385334e-14]
  [-3.35486434e-14 -3.83385334e-14  5.77454260e+00]]

 [[ 3.72360446e+00 -2.68267488e-15  7.55575892e-15]
  [-2.68267488e-15  3.72360446e+00  5.39748926e-15]
  [ 7.55575892e-15  5.39748926e-15  7.83948334e+00]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [38]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[-8.44721225e-06 -6.35221089e-12  1.21547943e-11]
  [-6.35221089e-12 -8.44719977e-06 -1.01864973e-11]
  [ 1.21547943e-11 -1.01864973e-11 -1.35753149e-05]]

 [[-5.69555470e-06 -4.38806362e-12  4.19090360e-12]
  [-4.38806362e-12 -5.69555557e-06 -5.82270256e-12]
  [ 4.19090360e-12 -5.82270256e-12 -3.91275105e-06]]]
